<a href="https://colab.research.google.com/github/Hoyadeath/practice_AI/blob/main/Practice_to_forecast_probability_of_Accident.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##MAIN

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from IPython.display import HTML

plt.rc('font', family='NamuGothic')
file_path = "/content/drive/MyDrive/Colab Notebooks/docs/accidentInfoList.CSV"
df = pd.read_csv(file_path, encoding='cp949')
df

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,2020010100100001,2020년 1월 1일 00시,수요일,서울특별시 양천구 목동,경상사고,0,0,1,0,차대사람 - 기타,...,맑음,단일로 - 기타,이륜,남,56세,상해없음,보행자,남,25세,경상
1,2020010100100002,2020년 1월 1일 00시,수요일,서울특별시 노원구 월계동,경상사고,0,0,1,0,차대차 - 측면충돌,...,흐림,교차로 - 교차로안,승용,남,21세,상해없음,승용,남,53세,경상
2,2020010100100015,2020년 1월 1일 01시,수요일,서울특별시 용산구 한남동,경상사고,0,0,3,0,차대차 - 후진중충돌,...,맑음,기타 - 기타,승용,남,37세,상해없음,승용,남,64세,상해없음
3,2020010100100016,2020년 1월 1일 01시,수요일,서울특별시 영등포구 영등포동3가,경상사고,0,0,1,0,차대사람 - 기타,...,맑음,단일로 - 기타,승용,남,54세,상해없음,보행자,남,31세,경상
4,2020010100100017,2020년 1월 1일 01시,수요일,서울특별시 구로구 구로동,중상사고,0,2,3,0,차대사람 - 횡단중,...,맑음,교차로 - 교차로안,승용,남,23세,상해없음,보행자,남,46세,중상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102833,2022123100100522,2022년 12월 31일 23시,토요일,서울특별시 광진구 군자동,경상사고,0,0,1,0,차대차 - 기타,...,맑음,단일로 - 기타,원동기,남,22세,상해없음,원동기,남,32세,경상
102834,2022123100100523,2022년 12월 31일 23시,토요일,서울특별시 마포구 염리동,경상사고,0,0,1,0,차대차 - 추돌,...,맑음,단일로 - 기타,승용,남,60세,상해없음,승용,남,56세,경상
102835,2022123100100524,2022년 12월 31일 23시,토요일,서울특별시 중랑구 면목동,부상신고사고,0,0,0,1,차대차 - 측면충돌,...,맑음,단일로 - 기타,승합,남,47세,상해없음,이륜,남,54세,부상신고
102836,2022123100100525,2022년 12월 31일 23시,토요일,서울특별시 구로구 구로동,경상사고,0,0,1,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로부근,이륜,남,47세,상해없음,이륜,남,32세,경상


In [32]:
df.head()

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,2020010100100001,2020년 1월 1일 00시,수요일,서울특별시 양천구 목동,경상사고,0,0,1,0,차대사람 - 기타,...,맑음,단일로 - 기타,이륜,남,56세,상해없음,보행자,남,25세,경상
1,2020010100100002,2020년 1월 1일 00시,수요일,서울특별시 노원구 월계동,경상사고,0,0,1,0,차대차 - 측면충돌,...,흐림,교차로 - 교차로안,승용,남,21세,상해없음,승용,남,53세,경상
2,2020010100100015,2020년 1월 1일 01시,수요일,서울특별시 용산구 한남동,경상사고,0,0,3,0,차대차 - 후진중충돌,...,맑음,기타 - 기타,승용,남,37세,상해없음,승용,남,64세,상해없음
3,2020010100100016,2020년 1월 1일 01시,수요일,서울특별시 영등포구 영등포동3가,경상사고,0,0,1,0,차대사람 - 기타,...,맑음,단일로 - 기타,승용,남,54세,상해없음,보행자,남,31세,경상
4,2020010100100017,2020년 1월 1일 01시,수요일,서울특별시 구로구 구로동,중상사고,0,2,3,0,차대사람 - 횡단중,...,맑음,교차로 - 교차로안,승용,남,23세,상해없음,보행자,남,46세,중상


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102838 entries, 0 to 102837
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   사고번호        102838 non-null  int64 
 1   사고일시        102838 non-null  object
 2   요일          102838 non-null  object
 3   시군구         102838 non-null  object
 4   사고내용        102838 non-null  object
 5   사망자수        102838 non-null  int64 
 6   중상자수        102838 non-null  int64 
 7   경상자수        102838 non-null  int64 
 8   부상신고자수      102838 non-null  int64 
 9   사고유형        102838 non-null  object
 10  법규위반        102838 non-null  object
 11  노면상태        102838 non-null  object
 12  기상상태        102838 non-null  object
 13  도로형태        102838 non-null  object
 14  가해운전자 차종    102838 non-null  object
 15  가해운전자 성별    102838 non-null  object
 16  가해운전자 연령    102838 non-null  object
 17  가해운전자 상해정도  102838 non-null  object
 18  피해운전자 차종    99885 non-null   object
 19  피해운전자 성별    99885 non-n

In [34]:
#Delete unused features
df.drop(columns=['사고번호'], inplace=True)
#Modify '사고일시'column (accident date)
df['사고일시'] = pd.to_datetime(df['사고일시'], format='%Y년 %m월 %d일 %H시')
#Leave only street information '시군구' feature as '구'
df['시군구'] = df['시군구'].apply(lambda x: x.split(' ')[1])\
# Create ECLO Feature
df['ECLO'] = df['사망자수']*10 + df['중상자수']*5 + df['경상자수']*3 + df['부상신고자수']*1

#Data imputation
df['피해운전자 차종'].fillna('NaN', inplace=True)
df['피해운전자 성별'].fillna('NaN', inplace=True)
df['피해운전자 상해정도'].fillna('NaN', inplace=True)


In [35]:
#Create Fuction for extract numbers
def extract_numbers_from_age(age):
    numbers = re.findall(r'\d+', str(age))
    if numbers:
        return int(numbers[0])
    else:
        return np.nan

#extract numbers only & delete string data
df['가해운전자 연령'] = df['가해운전자 연령'].apply(extract_numbers_from_age)
df['피해운전자 연령'] = df['피해운전자 연령'].apply(extract_numbers_from_age)

#Mean Imputation
mean_age_driver = round(df['가해운전자 연령'].mean(), 1)
mean_age_victim = round(df['피해운전자 연령'].mean(), 1)

df['가해운전자 연령'].fillna(mean_age_driver, inplace=True)
df['피해운전자 연령'].fillna(mean_age_victim, inplace=True)

##SUB

In [ ]:
#기초통계분석
basic_stats = df.describe(include='all', datetime_is_numeric=True)
html= basic_stats.to_html()
html= html.replace('class"table table-striped"', 'class="table table-bordered table-striped"')
display(HTML(html))

In [ ]:
print(df.columns.tolist())

In [ ]:
print(df.head())

In [ ]:
df.head()

In [40]:
#One hot encoding example
categorical_features = ['요일']
encoded_df = pd.get_dummies(df[categorical_features])
df = df.drop(columns=categorical_features)
df = pd.concat([df, encoded_df], axis=1)

In [47]:
# Overall Process
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

file_path = file_path = "/content/drive/MyDrive/Colab Notebooks/docs/accidentInfoList.CSV"
df = pd.read_csv(file_path, encoding='cp949')

#Processing categorical data
categorical_features = ['요일']
encoded_df = pd.get_dummies(df[categorical_features])
df = df.drop(columns=categorical_features)
df = pd.concat([df, encoded_df], axis=1)

#Feature engineering
df['ECLO'] = df['사망자수']*10 + df['중상자수']*5 + df['경상자수']*3 + df['부상신고자수']*1

#Select required features and targets
X = df[['요일_금요일','요일_목요일','요일_수요일','요일_월요일','요일_토요일','요일_화요일','요일_일요일']]
y = df['ECLO']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Create and train random forest Regression model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

#Model evaluation
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE", rmse)

RMSE 2.9209614133743234
